# Exploration of Paley-Wiener Thm

In this notebook, we briefly explore numerically the result of a variant of the Paley-Wiener theorem: if $f$ is analytic and $2\pi$-periodic, then there exists $\alpha > 0$ such that $|\hat{f}_k| \leq C e^{-\alpha |k|}$. The $\alpha$ can be chosen such that $f$ is analytic in the strip $\Omega_\alpha := \{ z + i y : |y| \leq \alpha \}$. 

In [ ]:
using Pkg; Pkg.activate(".")
using Plots, QuadGK, LinearAlgebra, LaTeXStrings
include("tools.jl")

In [ ]:
"""
This is a very naive method to compute the 
Fourier coefficients. We will soon learn about 
much more efficient schemes based on the FFT.
"""
function compute_fcoeffs(f_fun, N)
    fhat = zeros(ComplexF64, 2*N+1)
    kgrid = -N:N
    for (i, k) in enumerate(kgrid)
        g = x -> f_fun(x) * exp(-im * k * x)
        fhat[i] = quadgk(g, -pi, pi; rtol = 1e-6, atol=1e-6)[1] / (2*pi)
    end
    return fhat, kgrid
end

Our model problem is a periodic version of the Witch of Agnesi, 
$$
  f(x) = \frac{1}{1+ c^2 \sin^2(x)}
$$
It's analytic extension into the complex plane has a singularity 
at $z = \pm i \alpha$, where $\alpha = \sinh^{-1}(c^{-1})$.

In [ ]:
f, α = let c = 3
    (x -> 1 / (1 + c^2 * sin(x)^2)), asinh(1/c)
end

F̂, kgrid = compute_fcoeffs(f, 20);


In [ ]:
scatter(abs.(kgrid), abs.(F̂); ms = 6, 
        size = (400, 300), yscale = :log10, label = "", 
        xlabel = "|k|", ylims = (1e-4, 1))
plot!( [10, 18], 0.1*exp.(-α * [10, 18]), lw=2, ls=:dash, c=:black, 
        label = L"\exp(-\alpha N)")

We can quickly confirm the resulting rate of approximation.

In [ ]:
function eval_trig(x, uhat) 
    N = (length(uhat)-1) ÷ 2
    return real( sum(uhat[k+N+1] * exp(im * x * k) for k = -N:N) )
end

NN = [4, 8, 12, 16, 20, 24, 28]
xe = range(-π, π, length=1000)
f_ex = f.(xe)
err_max = zeros(length(NN))
for (i, N) in enumerate(NN)
    F̂, _ = compute_fcoeffs(f, N);
    fN = eval_trig.(xe, Ref(F̂))
    err_max[i] = norm(f_ex - fN, Inf)
end


In [ ]:
plot(NN, err_max; lw=3, m=:o, ms=5, label = L"\|f - \Pi_N f\|_\infty",
    yscale = :log10, xscale = :identity, xlabel = L"N", 
    size = (400, 300))
plot!(NN[4:end], exp.(-α * NN[4:end]); lw=2, ls=:dash, c = :black, 
        label = L"e^{-\alpha N}")